<a href="https://colab.research.google.com/github/BrunoReis136/AI_features/blob/main/ai_video_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# üß∞ Instalar depend√™ncias
!pip install diffusers transformers accelerate scipy safetensors opencv-python moviepy

# üß± Importa√ß√µes
import os
import cv2
from moviepy.editor import *
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from transformers import AutoTokenizer
import torch

# üìÅ Criar diret√≥rios
os.makedirs("frames_original", exist_ok=True)
os.makedirs("frames_transformados", exist_ok=True)

# üìΩÔ∏è Upload do v√≠deo
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# üéûÔ∏è Extrair frames
vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()
count = 0
while success:
    cv2.imwrite(f"frames_original/frame{count:03d}.png", image)
    success, image = vidcap.read()
    count += 1
print(f"{count} frames extra√≠dos.")

# üì¶ Baixar modelo ControlNet
from diffusers.utils import load_image
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
).to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# üé® Fun√ß√£o para transformar cada frame
from PIL import ImageFilter

prompt = input("\n Descreva as mudan√ßas no v√≠deo: \n")
negative_prompt = "blurry, low quality, deformed, text, watermark"

for i in range(count):
    img = Image.open(f"frames_original/frame{i:03d}.png")
    img = img.resize((512, 512))  # redimensiona para SD
    img = img.filter(ImageFilter.EDGE_ENHANCE)  # ou canny se quiser
    output = pipe(prompt, image=img, num_inference_steps=25, negative_prompt=negative_prompt).images[0]
    output.save(f"frames_transformados/frame{i:03d}.png")

# üéûÔ∏è Juntar os frames transformados em v√≠deo
image_files = [f"frames_transformados/frame{i:03d}.png" for i in range(count)]
clip = ImageSequenceClip(image_files, fps=vidcap.get(cv2.CAP_PROP_FPS))
clip.write_videofile("video_sinistro.mp4", codec="libx264")


In [ ]:
# üß∞ Instalar depend√™ncias
!pip install opencv-python moviepy

# üß± Importa√ß√µes
import os
import cv2
import numpy as np
from PIL import Image
from moviepy.editor import ImageSequenceClip
from google.colab import files

# üìÅ Criar diret√≥rios
os.makedirs("frames_original", exist_ok=True)
os.makedirs("frames_transformados", exist_ok=True)

# üì§ Upload do v√≠deo e personagem
print("Fa√ßa upload do V√çDEO e da IMAGEM PNG do personagem com fundo transparente.")
uploaded = files.upload()
video_path = [f for f in uploaded.keys() if f.endswith(".mp4")][0]
personagem_path = [f for f in uploaded.keys() if f.endswith(".png")][0]

# üéûÔ∏è Extrair frames do v√≠deo
vidcap = cv2.VideoCapture(video_path)
fps = vidcap.get(cv2.CAP_PROP_FPS)
success, image = vidcap.read()
count = 0
while success:
    cv2.imwrite(f"frames_original/frame{count:03d}.png", image)
    success, image = vidcap.read()
    count += 1
print(f"{count} frames extra√≠dos.")

# üßü‚Äç‚ôÇÔ∏è Carregar personagem com fundo transparente
personagem = Image.open(personagem_path).convert("RGBA")
personagem = personagem.resize((150, 300))  # ajuste conforme desejar

# üå´Ô∏è Fun√ß√£o para aplicar neblina no frame (OpenCV)
def aplicar_neblina_cv2(frame_np):
    h, w, _ = frame_np.shape
    neblina = np.random.normal(loc=200, scale=30, size=(h, w, 3)).astype(np.uint8)
    neblina = cv2.GaussianBlur(neblina, (101, 101), 0)
    return cv2.addWeighted(frame_np, 0.85, neblina, 0.15, 0)

# üñºÔ∏è Processar cada frame
for i in range(count):
    # Carregar frame original
    frame_path = f"frames_original/frame{i:03d}.png"
    frame = Image.open(frame_path).convert("RGBA")

    # Adicionar personagem (simula movimento vindo de longe)
    x = 200  # posi√ß√£o horizontal fixa
    y = 400 - i * 4  # personagem se aproxima
    frame_editado = frame.copy()
    frame_editado.paste(personagem, (x, y), personagem)

    # Converter para NumPy para aplicar neblina com OpenCV
    frame_np = np.array(frame_editado.convert("RGB"))
    frame_np = aplicar_neblina_cv2(frame_np)

    # Salvar frame final
    final = Image.fromarray(frame_np)
    final.save(f"frames_transformados/frame{i:03d}.png")

print("‚úÖ Todos os frames foram processados.")

# üìΩÔ∏è Recriar v√≠deo final
image_files = [f"frames_transformados/frame{i:03d}.png" for i in range(count)]
clip = ImageSequenceClip(image_files, fps=fps)
clip.write_videofile("video_sinistro_final.mp4", codec="libx264")

print("‚úÖ V√≠deo gerado com sucesso: video_sinistro_final.mp4")
